<a href="https://colab.research.google.com/github/Jeremy-su1/ai-algorithm/blob/main/final/single_cls_mamba2_130m.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch==2.1.1 torchvision==0.16.1 torchaudio==2.1.1 --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 105.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 88.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 24.3 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.5.0+cu121
    Uninstalling torch-2.5.0+cu121:
      Successfully uninstalled torch-2.5.0+cu121
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.20.0+cu121
    Uninstalling torchvision-0.20.0+cu121:
      Successfully uninstalled torchvision-0.20.0+cu121
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.5.0+cu121
    Uninstalling torchaudio-2.5.0+cu121:
      Successfully uninstalled torchaudio-2.5.0+cu121


In [2]:
!pip install causal-conv1d>=1.1.0

In [3]:
!pip install mamba-ssm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mamba-ssm: filename=mamba_ssm-2.2.2-cp310-cp310-linux_x86_64.whl size=343412612 sha256=150be68a91b3444b35ee2cda6a4790aaf83711009a8bf8e91afc7a316b091df6
  Stored in directory: /root/.cache/pip/wheels/57/7c/90/9f963468ecc3791e36e388f9e7b4a4e1e3f90fbb340055aa4d
Successfully built mamba-ssm


In [4]:
!pip install datasets evaluate accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [5]:
!export LC_ALL="en_US.UTF-8"
!export LD_LIBRARY_PATH="/usr/lib64-nvidia"
!export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
!ldconfig /usr/lib64-nvidia

/sbin/ldconfig.real: /usr/local/lib/libur_loader.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libumf.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtcm.so.1 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libhwloc.so.15 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtcm_debug.so.1 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libur_adapter_opencl.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libur_adapter_level_zero.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is

In [6]:
import os
import random
import json
import torch
import torch.nn as nn
from collections import namedtuple
from dataclasses import dataclass, field, asdict
from mamba_ssm.models.mixer_seq_simple import MambaLMHeadModel
from mamba_ssm.utils.hf import load_config_hf, load_state_dict_hf

import evaluate
import numpy as np
from datasets import load_dataset
from transformers import Trainer
from transformers import AutoTokenizer, TrainingArguments

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
from huggingface_hub import notebook_login
notebook_login()

In [9]:
import wandb
wandb.login(key="ee23d2708eaedb89d7020a87e3ac4dea2b742261")
wandb.init(project="Single label classification", entity="ai_expert", name="mamba2-130m")

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ebinna-lee (ai_expert). Use `wandb login --relogin` to force relogin


In [10]:
from dataclasses import dataclass, asdict
import json

@dataclass
class MambaConfig:
    d_model: int = 768
    d_intermediate: int = 0
    n_layer: int = 24
    vocab_size: int = 50277
    ssm_cfg: dict = None
    attn_layer_idx: list = None
    attn_cfg: dict = None
    rms_norm: bool = True
    residual_in_fp32: bool = True
    fused_add_norm: bool = True
    pad_vocab_size_multiple: int = 16
    tie_embeddings: bool = True

    def __post_init__(self):
        # 기본값으로 설정된 None 타입을 빈 딕셔너리와 빈 리스트로 초기화
        if self.ssm_cfg is None:
            self.ssm_cfg = {"layer": "Mamba2"}
        if self.attn_layer_idx is None:
            self.attn_layer_idx = []
        if self.attn_cfg is None:
            self.attn_cfg = {}

    def to_json_string(self):
        return json.dumps(asdict(self))

    def to_dict(self):
        return asdict(self)

In [11]:
classes = ['biology', 'cooking', 'diy', 'travel', 'stackoverflow']
class2id = {'biology' :0, 'cooking' : 1, 'diy' : 2, 'travel' : 3, 'stackoverflow' : 4}
id2class = {0 : 'biology', 1: 'cooking', 2 : 'diy', 3 : 'travel', 4 : 'stackoverflow'}

In [12]:
class MambaClassificationHead(nn.Module):
    def __init__(self, d_model, num_classes, **kwargs):
        super(MambaClassificationHead, self).__init__()
        self.classification_head = nn.Linear(d_model, num_classes, **kwargs)

    def forward(self, hidden_states):
        return self.classification_head(hidden_states)

In [13]:
class MambaTextClassification(MambaLMHeadModel):
    def __init__(
        self,
        config: MambaConfig,
        initializer_cfg=None,
        device=None,
        dtype=None,
    ) -> None:
        super().__init__(config, initializer_cfg, device, dtype)

        self.classification_head = MambaClassificationHead(d_model=config.d_model, num_classes=len(classes))
        del self.lm_head
        self.multi_label = False
        self.id2label = id2class
        self.class2id = class2id

    @classmethod
    def addMambaClassificationHead(cls, num_classes, id2label, class2id, multi_label):
        cls.classification_head = MambaClassificationHead
        del self.lm_head

    def forward(self, input_ids, attention_mask=None, labels=None):
        hidden_states = self.backbone(input_ids)
        mean_hidden_states = hidden_states.mean(dim=1)

        logits = self.classification_head(mean_hidden_states)
        if labels is None:
          ClassificationOutput = namedtuple("ClassificationOutput", ["logits"])
          return ClassificationOutput(logits=logits)
        else:
          ClassificationOutput = namedtuple("ClassificationOutput", ["loss", "logits"])
          if self.multi_label:
            loss_fct = nn.BCEWithLogitsLoss()
            loss = loss_fct(logits, labels)
          else:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits, labels)
          return ClassificationOutput(loss=loss, logits=logits)

    def predict(self, text, tokenizer):
        input_ids = torch.tensor(tokenizer(text)['input_ids'], device='cuda')[None]
        with torch.no_grad():
          logits = self.forward(input_ids).logits[0]

        if self.multi_label:
          probabilities = torch.sigmoid(logits).cpu().numpy()
          predictions = (probabilities > 0.5).astype(int)
          return [self.id2label[i] for i, value in enumerate(predictions) if value == 1]
        else:
          label = np.argmax(logits.cpu().numpy())
          return self.id2label[label]

    @classmethod
    def from_pretrained(cls, pretrained_model_name, device=None, dtype=None, **kwargs):
        config_data = load_config_hf(pretrained_model_name)
        config = MambaConfig(**config_data)

        model = cls(config, device=device, dtype=dtype, **kwargs)

        model_state_dict = load_state_dict_hf(pretrained_model_name, device=device, dtype=dtype)
        model.load_state_dict(model_state_dict, strict=False)

        print("Newly initialized embedding:", set(model.state_dict().keys()) - set(model_state_dict.keys()))
        return model

In [14]:
model = MambaTextClassification.from_pretrained("state-spaces/mamba2-130m")
model.to("cuda")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/330 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/258M [00:00<?, ?B/s]

Newly initialized embedding: {'classification_head.classification_head.bias', 'classification_head.classification_head.weight'}


MambaTextClassification(
  (backbone): MixerModel(
    (embedding): Embedding(50288, 768)
    (layers): ModuleList(
      (0-23): 24 x Block(
        (norm): RMSNorm()
        (mixer): Mamba2(
          (in_proj): Linear(in_features=768, out_features=3352, bias=False)
          (conv1d): Conv1d(1792, 1792, kernel_size=(4,), stride=(1,), padding=(3,), groups=1792)
          (act): SiLU()
          (norm): RMSNorm()
          (out_proj): Linear(in_features=1536, out_features=768, bias=False)
        )
      )
    )
    (norm_f): RMSNorm()
  )
  (classification_head): MambaClassificationHead(
    (classification_head): Linear(in_features=768, out_features=5, bias=True)
  )
)

In [15]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")
tokenizer.pad_token_id = tokenizer.eos_token_id

tokenizer_config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/457k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [17]:
from datasets import load_dataset

dataset_train = load_dataset(
    'csv',
    data_files='/content/drive/My Drive/AiExpertCource/project/dataset/stackexchage/stackexchange_train_dataset_10000_title.csv',
    split='train'
)
dataset_valid = load_dataset(
    'csv',
    data_files='/content/drive/My Drive/AiExpertCource/project/dataset/stackexchage/stackexchange_val_dataset_1000_title.csv',
    split='train'
)
dataset_test = load_dataset(
    'csv',
    data_files='/content/drive/My Drive/AiExpertCource/project/dataset/stackexchage/stackexchange_test_dataset_1000_title.csv',
    split='train'
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [18]:
import ast

def preprocess_function(example):
   samples = tokenizer(example['title_content'], truncation=True, max_length=2048)
   samples.pop('attention_mask')
   return samples

In [19]:
tokenized_train_dataset = dataset_train.map(preprocess_function)
tokenized_valid_dataset = dataset_valid.map(preprocess_function)
tokenized_test_dataset = dataset_test.map(preprocess_function)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [20]:
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["title_content"])
tokenized_valid_dataset = tokenized_valid_dataset.remove_columns(["title_content"])
tokenized_test_dataset = tokenized_test_dataset.remove_columns(["title_content"])

In [21]:
accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    predictions = np.argmax(predictions, axis=1)

    return {
        "accuracy": accuracy.compute(predictions=predictions, references=labels)["accuracy"],
        "micro_precision": precision.compute(predictions=predictions, references=labels, average="micro")["precision"],
        "micro_recall": recall.compute(predictions=predictions, references=labels, average="micro")["recall"],
        "micro_f1": f1.compute(predictions=predictions, references=labels, average="micro")["f1"],
        "macro_precision": precision.compute(predictions=predictions, references=labels, average="macro")["precision"],
        "macro_recall": recall.compute(predictions=predictions, references=labels, average="macro")["recall"],
        "macro_f1": f1.compute(predictions=predictions, references=labels, average="macro")["f1"],
        "weighted_precision": precision.compute(predictions=predictions, references=labels, average="weighted")["precision"],
        "weighted_recall": recall.compute(predictions=predictions, references=labels, average="weighted")["recall"],
        "weighted_f1": f1.compute(predictions=predictions, references=labels, average="weighted")["f1"],
    }

In [22]:
class MambaTrainer(Trainer):

    def compute_loss(self, model, inputs, return_outputs=False):
        input_ids = inputs.pop("input_ids")
        labels = inputs.pop('labels')
        outputs = model(input_ids=input_ids, labels=labels)
        loss = outputs.loss

        return (loss, outputs) if return_outputs else loss

    def save_model(self, output_dir = None, _internal_call = False):
        if output_dir is None:
            output_dir = self.args.output_dir

        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        torch.save(self.model.state_dict(), f"{output_dir}/pytorch_model.bin")

        self.tokenizer.save_pretrained(output_dir)

        with open(f'{output_dir}/config.json', 'w') as f:
            json.dump(self.model.config.to_dict(), f)

In [23]:
output_dir = "single_cls_mamba2-130m"

In [24]:
training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    push_to_hub=True,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [25]:
trainer = MambaTrainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_valid_dataset,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics
    )

In [26]:
text =  """Can't get read to work properly in this script
  I have the following script:

echo "User already exists, delete user? [Y/n] "
read -r -n 1 yn
echo "$yn was pressed"
When I run the script:

Pressing y or Y (without [Enter]) echoes the character to the screen.
Pressing [Enter] moves the cursor to a new line, but the following echo command doesn't execute
Pressing [Enter] a second time causes the echo command to execute
I expect it to not require me to press [Enter] twice.

I have tried different variations including putting the prompt in the read command, but that was worse since the prompt never even showed up at that point.

What am I doing wrong?
});
"""

In [27]:
response = model.predict(text, tokenizer)
print(response)

cooking


In [28]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Accuracy,Micro Precision,Micro Recall,Micro F1,Macro Precision,Macro Recall,Macro F1,Weighted Precision,Weighted Recall,Weighted F1
1,0.148300,0.138470,0.965000,0.965000,0.965000,0.965000,0.966230,0.965000,0.965016,0.966230,0.965000,0.965016
2,0.035000,0.141276,0.977000,0.977000,0.977000,0.977000,0.977392,0.977000,0.977035,0.977392,0.977000,0.977035
3,0.022600,0.144151,0.974000,0.974000,0.974000,0.974000,0.974121,0.974000,0.973991,0.974121,0.974000,0.973991
4,0.003500,0.142762,0.976000,0.976000,0.976000,0.976000,0.976023,0.976000,0.975992,0.976023,0.976000,0.975992
5,0.002400,0.147171,0.977000,0.977000,0.977000,0.977000,0.977037,0.977000,0.976989,0.977037,0.977000,0.976989


/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


TrainOutput(global_step=6250, training_loss=0.05843438465476036, metrics={'train_runtime': 1236.3892, 'train_samples_per_second': 40.44, 'train_steps_per_second': 5.055, 'total_flos': 0.0, 'train_loss': 0.05843438465476036, 'epoch': 5.0})

In [29]:
trainer.evaluate()

{'eval_loss': 0.1412755697965622,
 'eval_accuracy': 0.977,
 'eval_micro_precision': 0.977,
 'eval_micro_recall': 0.977,
 'eval_micro_f1': 0.977,
 'eval_macro_precision': 0.9773916663139428,
 'eval_macro_recall': 0.977,
 'eval_macro_f1': 0.9770353242750016,
 'eval_weighted_precision': 0.9773916663139428,
 'eval_weighted_recall': 0.977,
 'eval_weighted_f1': 0.9770353242750015,
 'eval_runtime': 5.8855,
 'eval_samples_per_second': 169.91,
 'eval_steps_per_second': 21.239,
 'epoch': 5.0}

In [30]:
trainer_for_test = MambaTrainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics
    )

In [31]:
trainer_for_test.evaluate()

{'eval_loss': 0.1846182495355606,
 'eval_model_preparation_time': 0.0033,
 'eval_accuracy': 0.968,
 'eval_micro_precision': 0.968,
 'eval_micro_recall': 0.968,
 'eval_micro_f1': 0.968,
 'eval_macro_precision': 0.968499783371348,
 'eval_macro_recall': 0.968,
 'eval_macro_f1': 0.9680164756565887,
 'eval_weighted_precision': 0.968499783371348,
 'eval_weighted_recall': 0.968,
 'eval_weighted_f1': 0.9680164756565887,
 'eval_runtime': 5.8982,
 'eval_samples_per_second': 169.544,
 'eval_steps_per_second': 21.193}

In [32]:
trainer.push_to_hub(commit_message="Training complete")

events.out.tfevents.1731248401.775cb8dac9de.2521.1:   0%|          | 0.00/921 [00:00<?, ?B/s]

events.out.tfevents.1731247149.775cb8dac9de.2521.0:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1731248407.775cb8dac9de.2521.2:   0%|          | 0.00/892 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ebinna/single_cls_mamba2-130m/commit/7872412b000b25790ac0c6800b7929c67b09f921', commit_message='Training complete', commit_description='', oid='7872412b000b25790ac0c6800b7929c67b09f921', pr_url=None, pr_revision=None, pr_num=None)

In [33]:
response = model.predict(text, tokenizer)
print(response)

stackoverflow


In [34]:
ebinna = MambaTextClassification.from_pretrained("ebinna/"+output_dir)
ebinna.to("cuda")

tk = AutoTokenizer.from_pretrained("ebinna/"+output_dir)
tk.pad_token_id = tk.eos_token_id

config.json:   0%|          | 0.00/266 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/516M [00:00<?, ?B/s]

Newly initialized embedding: set()


tokenizer_config.json:   0%|          | 0.00/4.84k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

In [35]:
res = ebinna.predict(text, tk)
print(res)

stackoverflow


In [36]:
wandb.finish()

eval/accuracy,▁█▆▇██▃
eval/loss,▁▁▂▂▂▁█
eval/macro_f1,▁█▆▇██▃
eval/macro_precision,▁█▆▇██▂
eval/macro_recall,▁█▆▇██▃
eval/micro_f1,▁█▆▇██▃
eval/micro_precision,▁█▆▇██▃
eval/micro_recall,▁█▆▇██▃
eval/model_preparation_time,▁
eval/runtime,▂▃▁▄█▃▃
eval/samples_per_second,▇▆█▅▁▆▅
